# Evaluate relative fitness from trait distributions

### Parameters of this notebook 

In [ ]:
import pandas as pd           

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from scipy import integrate
from scipy import stats
import random


In [ ]:
### Update dependent parameters according to input
import os
import os.path
from os import path

## create export directory if necessary
## foldernames for output plots/lists produced in this notebook
import os
FIG_DIR = f'./figures/handcrafted_correlation/'
os.makedirs(FIG_DIR, exist_ok=True)
print("All  plots will be stored in: \n" + FIG_DIR)

In [ ]:


### execute script to load modules here
exec(open('setup_aesthetics.py').read())

In [ ]:
DATASET_COLOR = 'darkorange'

In [ ]:
SUFFIX_DATASET = 'pleiotropic_handcrafted'

### Define initial condition for bulk growth cycle

In [ ]:
### set initial resource concentrations

CONCENTRATION_GLUCOSE = 20/180 * 1e3 # concentrations are recored  in milliMolar, to match the units of yield
print(CONCENTRATION_GLUCOSE)

In [ ]:
### define default initial_OD
OD_FACTOR = 1
OD_START = 0.01 *OD_FACTOR  #want OD_START to match the warringer data


In [ ]:
INITIAL_FREQ = 0.5

### Create trait distribution

In [ ]:
### handcrafted example scaled to larger number of points

mean_wt = (1,2,1./CONCENTRATION_GLUCOSE) #gmax, lag ,yield
NO_CURVES = 100
def lag2gmax(lag):
    return 0.1*(lag+mean_wt[1]) - 0.2 + mean_wt[0]


effect_lag = np.linspace(-1.2, -0.8, num = NO_CURVES)
mutant_lag = mean_wt[1] + effect_lag
mutant_gmax = lag2gmax(mutant_lag)

def gmax2yield(gmax):
    ## define two constants
    c = np.log(3)/0.01
    A = np.exp(-c*1.1)*OD_FACTOR/CONCENTRATION_GLUCOSE # use same units as empirical data
    return A*np.exp(c*(gmax))

mutant_yield = gmax2yield(mutant_gmax)

effect_sample = np.vstack([mutant_gmax, mutant_lag, mutant_yield]).T


In [ ]:
FIG_DIR_DATASET = FIG_DIR
os.makedirs(FIG_DIR_DATASET, exist_ok=True)

os.makedirs('./output/synthetic/', exist_ok=True)
EXPORT_DATASET = './output/synthetic/' + SUFFIX_DATASET + '.csv'


In [ ]:
### set up as dataframe

## create dataframe with mutant effects
df_effects = pd.DataFrame(data=effect_sample, columns = ['gmax', 'lag', 'yield'])
df_effects['is_wildtype'] = False

mean_wildtype = pd.Series(data =mean_wt, index =['gmax', 'lag', 'yield'] )
mean_wildtype['is_wildtype'] = True

df_effects=df_effects.append(mean_wildtype, ignore_index=True)

In [ ]:
def row2label(row):
    if row['is_wildtype'] == True:
        return 'wild-type'
    else:
        return 'knockout'

df_effects['label']  = df_effects.apply(row2label,axis=1)

In [ ]:

df_traits = df_effects
df_traits.to_csv(EXPORT_DATASET, index = False, float_format= '%.6e')

### plot trait distributionn

In [ ]:
from scipy.stats import spearmanr, pearsonr

In [ ]:
from latex_format import float2latex

In [ ]:

n_datapoints = df_traits.shape[0]
is_wildtype = df_traits['is_wildtype']==True


In [ ]:
palette = {'wild-type':'orange', 'knockout': 'dimgrey'}

In [ ]:
### prepare data

is_wildtype =df_traits['is_wildtype'] == True 
df_knockouts = df_traits.loc[~is_wildtype]
data = df_knockouts.mask(is_wildtype, other = np.nan)

In [ ]:
xvar = 'gmax'
yvar = 'lag'

ratio = 5
grid = sns.jointplot(data=data, x=xvar, y=yvar, 
                     hue = 'label', palette = palette, legend = False,
                     marginal_kws = {'multiple':'layer', 'fill':False},
                     marginal_ticks= False, space = 0, ratio = ratio,
                     height = (1+1/ratio)*FIGHEIGHT_TRIPLET, )

ax = grid.ax_joint
ax = sns.scatterplot(data=data, x=xvar, y=yvar, color = 'dimgrey', edgecolor = 'none', ax = ax, legend=False)

## plot wild-type
x = mean_wildtype[xvar]
y = mean_wildtype[yvar]
ax.scatter(x=x,y=y, color = palette['wild-type'], label = 'wild-type')
#grid.ax_marg_y.axhline(y, color = palette['wild-type'])
#grid.ax_marg_x.axvline(x, color = palette['wild-type'])


### replot the marginal distributions in different colors
ax = grid.ax_marg_x
palette['knockout'] = 'dimgrey' ### fix the color for marginals in growth rate
sns.kdeplot(data = df_traits, x=xvar, ax=ax, fill = True, hue = 'label', palette=palette, multiple = 'layer',
       legend = False, cut = 0)

### replot the marginal distributions in different colors
ax = grid.ax_marg_y
palette['knockout'] = 'dimgrey' ### fix the color for marginals in growth rate
sns.kdeplot(data = df_traits, y=yvar, ax=ax, fill = True, hue = 'label', palette=palette, multiple = 'layer',
       legend = False, cut = 0)

## set label
ax = grid.ax_joint
ax.set_ylabel('lag time [hours]')
ax.set_xlabel('growth rate [per hour]')
## set legend
#ax.legend(loc = 'lefleft', bbox_to_anchor = (-0.65,0.99), frameon=False,
#          title = 'lag time [hours]', title_fontsize = MEDIUM_SIZE)
#ax.legend(frameon = False)
## set title

title = f"n={df_knockouts.shape[0]} mutants"
ax = grid.ax_marg_x
#ax.annotate(title, (0.7,0.05), xycoords ='axes fraction') # right
ax.annotate(title, (0.02,0.05), xycoords ='axes fraction') # left
grid.fig.savefig(FIG_DIR_DATASET + f"correlation_{xvar}-vs-{yvar}.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

In [ ]:
xvar = 'gmax'
yvar = 'yield'

ratio = 5
grid = sns.jointplot(data=data, x=xvar, y=yvar, 
                     hue = 'label', palette = palette, legend = False,
                     marginal_kws = {'multiple':'layer', 'fill':False},
                     marginal_ticks= False, space = 0, ratio = ratio,
                    height = (1+1/ratio)*FIGHEIGHT_TRIPLET, )


## replot with coloring by lag time
ax = grid.ax_joint
ax = sns.scatterplot(data=data, x=xvar, y=yvar, color = 'dimgrey', edgecolor = 'none', ax = ax, legend=False)

## plot wild-type
x = mean_wildtype[xvar]
y = mean_wildtype[yvar]
ax.scatter(x=x,y=y, color = palette['wild-type'], label = 'wild-type')
#grid.ax_marg_y.axhline(y, color = palette['wild-type'])
#grid.ax_marg_x.axvline(x, color = palette['wild-type'])


### replot the marginal distributions in different colors
ax = grid.ax_marg_x
palette['knockout'] = 'dimgrey' ### fix the color for marginals in growth rate
sns.kdeplot(data = df_traits, x=xvar, ax=ax, fill = True, hue = 'label', palette=palette, multiple = 'layer',
       legend = False, cut = 0)

### replot the marginal distributions in different colors
ax = grid.ax_marg_y
palette['knockout'] = 'dimgrey' ### fix the color for marginals in growth rate
sns.kdeplot(data = df_traits, y=yvar, ax=ax, fill = True, hue = 'label', palette=palette, multiple = 'layer',
       legend = False, cut= 0)

## fix y-axis limit, yield can only be positive
ax.set_ylim(-0.0004)

## set label
ax = grid.ax_joint
ax.set_ylabel('biomass yield [OD/mM glucose]')
ax.set_xlabel('growth rate [per hour]')
## set legend
#ax.legend(loc = 'lefleft', bbox_to_anchor = (-0.65,0.99), frameon=False,
#          title = 'lag time [hours]', title_fontsize = MEDIUM_SIZE)
#ax.legend(frameon = False)
## set title

title = f"n={df_knockouts.shape[0]} mutants"
ax = grid.ax_marg_x
#ax.annotate(title, (0.7,0.05), xycoords ='axes fraction') # right
ax.annotate(title, (0.02,0.05), xycoords ='axes fraction') # left
grid.fig.savefig(FIG_DIR_DATASET + f"correlation_{xvar}-vs-{yvar}.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)

### Load trait data into the standard form required by Michaels code

In [ ]:
# convert notation, so we can reuse old code
df_input = df_knockouts
n_input = df_input.shape[0]

In [ ]:
# create wildtype
WILDTYPE = df_traits[df_traits['label']=='wild-type']

In [ ]:
### growth rates
gs = np.zeros(n_input+1)
gs[0] = WILDTYPE['gmax']
gs[1:] = df_input['gmax'].values

### lag times
ls = np.zeros(n_input+1)
ls[0] = WILDTYPE['lag']
ls[1:] = df_input['lag'].values

### yield
Ys = np.zeros(n_input+1)
Ys[0] = WILDTYPE['yield']
Ys[1:] = df_input['yield'].values


### Calculate effective yield

In [ ]:
from bulk_simulation_code import CalcRelativeYield

In [ ]:
### calculcate effective yields
nus = CalcRelativeYield(Ys, R0 = CONCENTRATION_GLUCOSE, N0 = OD_START)


### Simulate pairwise competition growth cycles (scenario A)

In [ ]:
from bulk_simulation_code import toPerGeneration, run_pairwise_experiment
from bulk_simulation_code import CalcTotalSelectionCoefficientLogit

In [ ]:
%%time
xs_pair, xs_pair_final, tsats,fcs_both, fcs_wt, fcs_mut = run_pairwise_experiment(
                                                                gs=gs,   ls=ls,   nus = nus, 
                                                                g1=gs[0],l1=ls[0],nu1=nus[0],
                                                                x0 = INITIAL_FREQ)

In [ ]:
s_percycle = CalcTotalSelectionCoefficientLogit(xs_pair,xs_pair_final)
s_pergen = np.divide(s_percycle, np.log(fcs_wt))

### store results

In [ ]:
df_output = df_input.copy()

In [ ]:
df_output['logfc_wt'] = np.log(fcs_wt[1:])
df_output['logfc_mut'] = np.log(fcs_mut[1:])

df_output['logit_percycle'] = s_percycle[1:]
df_output['logit_pergen'] = s_pergen[1:]

df_output['logit_percycle_rank'] =df_output['logit_percycle'].rank()
df_output['logit_pergen_rank'] = df_output['logit_pergen'].rank()

df_output['deltarank'] = df_output['logit_pergen_rank'] - df_output['logit_percycle_rank'] 


### Prepare data for plotting

In [ ]:
### sort by label prepare for plotting

def row2label(row):
    if row['is_wildtype'] == True:
        return 'wild-type'
    else:
        return 'knockout'
    

In [ ]:
df_output['label']  = df_output.apply(row2label,axis=1)

df_output = df_output.sort_values('label')

In [ ]:
## sort by misranking

df_output['deltarank_abs'] = np.abs(df_output['deltarank'])
df_sorted = df_output.sort_values('deltarank', ascending = True)
select = df_sorted.index[[-1]]

In [ ]:
df_sorted.loc[select]

### plot misranking

In [ ]:
### plot residuals

palette = {'wild-type':'orange', 'knockout': 'dimgrey'}


fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET, FIGHEIGHT_TRIPLET))

x_var = 'logit_percycle'
y_var = 'deltarank'
data = df_output
sns.scatterplot(data = data, x = x_var, y = y_var, rasterized = False, ax = ax, 
                color = 'dimgrey', edgecolor = 'none', legend =False)


### plot select points
is_labeled = True
for i in select:
    A, B = float(data.loc[i, x_var]), float(data.loc[i, y_var])
    #ax.scatter(A-0.15,B,s=150,color ='tab:red', zorder = -1, marker = 5 )
    if is_labeled == False: 
        label = 'max. disagreement' 
        is_labeled = True
    else: label = None
    #ax.scatter(A,B,s=200,color ='tab:blue', zorder = -1,label = label, alpha = 0.25)


### plot horizontal line for orientation
ax.axhline(0,ls = '--', color = 'black')

### annotate
ax.set_xlabel('relative fitness per-cycle:' + r'  $s^{\mathrm{logit}}_{\mathrm{cycle}}$')
#ax.set_ylabel('rank difference to\nrelative fitness per-generation [rank]')
ax.set_ylabel('rank difference between fitness\nper-generation and fitness per-cycle')
ax.legend(loc = 'upper left', bbox_to_anchor = (-0.05,1.0), frameon=False) #inside

title = f"n = {sum(data['is_wildtype']==False)} mutants"
ax.set_title(title, loc = 'left')

fig.savefig(FIG_DIR + f"residuals_{x_var}_vs_{y_var}_x0={INITIAL_FREQ:.2f}.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)


### plot on foldchange phase diagram

In [ ]:
def eval_isocline_percycle(logfc_wt, level):
    return logfc_wt + level

def eval_isocline_pergen(logfc_wt, level):
    return np.multiply((level+1),logfc_wt)

In [ ]:

palette = {'wild-type':'orange', 'knockout': 'dimgrey'}

In [ ]:
### plot cloud of points

fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET, FIGHEIGHT_TRIPLET))

x_var = 'logfc_wt'
y_var = 'logfc_mut'
data = df_output
sns.scatterplot(data = data, x = x_var, y = y_var, rasterized = False, ax = ax, 
                color = 'dimgrey', zorder = 2, edgecolor = 'none', legend = False)



## find value limits

fcmax = np.max([data[x_var].max(),data[y_var].max()])
fcmin = np.min([data[x_var].min(),data[y_var].min()])
assert fcmin > 0

### set axis limits
#xmin = 1
#assert xmin < fcmin, 'We are cutting points from the dataplot!'
#xmax = 5.5
#assert xmax > fcmax, 'We are cutting points from the dataplot!'
#ax.set_xlim(xmin,xmax)
#ax.set_ylim(xmin,xmax)


## take off axis spines
#sns.despine(left=False, bottom = False, ax = ax)

### find axis limits
xmin, xmax = ax.get_xlim()
fcwt_vec = np.linspace(xmin,xmax, num = 100) 
fcwt_vec = np.concatenate((-fcwt_vec,fcwt_vec))
color_percycle = 'tab:grey'
color_pergen = 'navy'

### plot per cycle isoclines
levels = np.outer([-1,1],np.linspace(0.01,8,num = 6)).flatten()
levels.sort()

for level in levels: 

    y = eval_isocline_percycle(fcwt_vec, level = level)
    #ax.plot(fcwt_vec, y, color = color_percycle)
    
    
## plot per generationa isoclines
angles = np.linspace(0,np.pi/2 - 0.001, num = 6)
levels = np.outer([-1,1],np.tan(angles)).flatten()

for level in levels: 
    y = eval_isocline_pergen(fcwt_vec, level =level) 
    #ax.plot(fcwt_vec, y, color = color_pergen)
    
## plot diagonal 
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()
ax.plot([-xmin,xmax],[-xmin,xmax], color = 'black', ls = '--')
ax.set_xlim(xmin,xmax)
ax.set_ylim(ymin,ymax)
## add  legend items
#ax.plot([],[], color = color_percycle, label = 'per-cycle $s$ isocline')
#ax.plot([],[], color = color_pergen, label = 'per-generation $Q$ isocline')

## plot red cone for a select point
select = [0]
for i in select:
    A, B = float(data.loc[i, x_var]), float(data.loc[i, y_var])
    #ax.scatter(A,B,s=70,color ='tab:red', zorder = 3)
    #ax.scatter(A-0.05,B,s=150,color ='tab:red', zorder = -1, marker = 5 )
    #ax.scatter(A,B,s=70,color ='tab:red', zorder = 3, marker = 'v')
    #ax.scatter(A,B,s=200,color ='tab:blue', zorder = -1,label = label, alpha = 0.25)


    x_fill = np.linspace(fcwt_vec[0],fcwt_vec[-1])
    y_fill = B/A*x_fill

    ax.fill_between(x_fill, (x_fill - A) + B, y_fill, color='tab:red', alpha=0.25)


### annotate
#ax.legend(loc = 'upper left', bbox_to_anchor = (1.3,1)) # outside
#ax.legend(loc = 'upper left', bbox_to_anchor = (-0.05,0.25), frameon=False) #inside
title = f"n = {sum(data['is_wildtype']==False)} mutants"
#ax.set_title(title, loc = 'left')

ax.set_xlabel(r"wild-type log fold-change: $\mathrm{LFC}_{\mathrm{wt}}$")
ax.set_ylabel(r"mutant log fold-change: $\mathrm{LFC}_{\mathrm{mut}}$")

title = f"n = {sum(data['is_wildtype']==False)} mutants"
ax.set_title(title, loc = 'left')


fig.savefig(FIG_DIR+ f'scatterplot_logfc_wt_vs_logfc_mut_x0={INITIAL_FREQ:.2f}.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)
              

### plot DFEs

In [ ]:
import warnings

In [ ]:
palette = {'wild-type':'orange', 'knockout': 'dimgrey', 'wild-type median':'navy'}

In [ ]:
fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET, FIGHEIGHT_TRIPLET))

data = df_output
x_var = 'logit_percycle'
#sns.histplot(data, x = 'logit_percycle', rasterized = False, ax = ax,
#                hue = 'label', palette = palette)

## need a context wrapper, else pandas throws a Future Warning
## see https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    # Warning-causing lines of code here
    sns.kdeplot(data=data, x=x_var, hue="label",common_norm = True,
            palette = palette, multiple="layer", ax = ax, fill = True, legend = False)
    
### plot selection coefficient zero for orientation
#ax.axvline(0, ls = '--', color = 'black')
#ax.legend_.set_title('')
#ax.legend([],[])

title = f"n = {sum(data['is_wildtype']==False)} mutants"
ax.set_title(title, loc = 'left')

ax.set_xlabel('relative fitness per-cycle:' + r'  $s^{\mathrm{logit}}_{\mathrm{cycle}}$')
ax.set_ylabel('mutant density')


fig.savefig(FIG_DIR+ f'dfeplot_logit_percycle_x0={INITIAL_FREQ:.2f}.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)
    

In [ ]:
fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET, FIGHEIGHT_TRIPLET))

data = df_output
x_var = 'logit_pergen'
#sns.histplot(data, x = 'logit_percycle', rasterized = False, ax = ax,
#                hue = 'label', palette = palette)

## need a context wrapper, else pandas throws a Future Warning
## see https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    # Warning-causing lines of code here
    sns.kdeplot(data=data, x=x_var, hue="label",common_norm = True,
            palette = palette, multiple="layer", ax = ax, fill = True, legend = False)
    
### plot selection coefficient zero for orientation
#ax.axvline(0, ls = '--', color = 'black')
#ax.legend_.set_title('')
#ax.legend([],[])

title = f"n = {sum(data['is_wildtype']==False)} mutants"
ax.set_title(title, loc = 'left')

ax.set_xlabel('relative fitness per-generation:' + r'  $s^{\mathrm{logit}}_{\mathrm{gen}}$')
ax.set_ylabel('mutant density')


fig.savefig(FIG_DIR+ f'dfeplot_logit_pergen_x0={INITIAL_FREQ:.2f}.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)
    

### plot global correlation in ranks

In [ ]:
data.columns

In [ ]:
### plot correlation

fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET, FIGHEIGHT_TRIPLET))

x_var = 'logit_percycle_rank'
y_var = 'logit_pergen_rank'
data = df_output
sns.scatterplot(data = data, x = x_var, y = y_var, rasterized = False, ax = ax, 
                color = 'dimgrey', edgecolor = 'none', legend = False)

### plot select points
is_labeled = True
for i in select:
    A, B = float(data.loc[i, x_var]), float(data.loc[i, y_var])
    #ax.scatter(A-0.15,B,s=150,color ='tab:red', zorder = -1, marker = 5 )
    if is_labeled == False: 
        label = 'max. disagreement' 
        is_labeled = True
    else: label = None
    ax.scatter(A,B,s=200,color ='tab:blue', zorder = -1,label = label, alpha = 0.25)


### annotate
ax.legend(loc = 'upper left', bbox_to_anchor = (-0.05,1.0), frameon=False) #inside

ax.set_ylabel('relative fitness per-generation [rank]')
ax.set_xlabel('relative fitness per-cycle [rank]')

title = f"n = {sum(data['is_wildtype']==False)} mutants"
ax.set_title(title, loc = 'left')

fig.savefig(FIG_DIR + f"scatterplot_{x_var}_vs_{y_var}_x0={INITIAL_FREQ:.2f}.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)


### plot global correlation in values

In [ ]:
### plot correlation

fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET, FIGHEIGHT_TRIPLET))

x_var = 'logit_percycle'
y_var = 'logit_pergen'
data = df_output
sns.scatterplot(data = data, x = x_var, y = y_var, rasterized = False, ax = ax,
                color = 'dimgrey', edgecolor = 'none',  legend = False)

### plot select points
is_labeled = True
for i in select:
    A, B = float(data.loc[i, x_var]), float(data.loc[i, y_var])
    #ax.scatter(A-0.15,B,s=150,color ='tab:red', zorder = -1, marker = 5 )
    if is_labeled == False: 
        label = 'max. disagreement' 
        is_labeled = True
    else: label = None
    #ax.scatter(A,B,s=200,color ='tab:blue', zorder = -1,label = label, alpha = 0.25)


### annotate
ax.legend(loc = 'upper left', bbox_to_anchor = (-0.05,1.0), frameon=False) #inside

ax.set_xlabel('relative fitness per-cycle:' + r'  $s^{\mathrm{logit}}_{\mathrm{cycle}}$')
ax.set_ylabel('relative fitness per-generation:' + r'  $s^{\mathrm{logit}}_{\mathrm{gen}}$')

title = f"n = {sum(data['is_wildtype']==False)} mutants"
ax.set_title(title, loc = 'left')

fig.savefig(FIG_DIR + f"scatterplot_{x_var}_vs_{y_var}_x0={INITIAL_FREQ:.2f}.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)


## Check correlation with saturation time

In [ ]:
### plot correlation

fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET, FIGHEIGHT_TRIPLET))

x_var = 'logfc_wt'
y_var = 'logit_percycle'
data = df_output
sns.scatterplot(data = data, x = x_var, y = y_var, rasterized = False, ax = ax,
                color = 'dimgrey', edgecolor = 'none', legend = False)

### plot select points
is_labeled = True
for i in select:
    A, B = float(data.loc[i, x_var]), float(data.loc[i, y_var])
    #ax.scatter(A-0.15,B,s=150,color ='tab:red', zorder = -1, marker = 5 )
    if is_labeled == False: 
        label = 'max. disagreement' 
        is_labeled = True
    else: label = None
    #ax.scatter(A,B,s=200,color ='tab:blue', zorder = -1,label = label, alpha = 0.25)


### annotate
#ax.legend(loc = 'upper left', bbox_to_anchor = (-0.05,1.0), frameon=False) #inside

ax.set_ylabel('relative fitness per-cycle:' + r'  $s^{\mathrm{logit}}_{\mathrm{cycle}}$')
ax.set_xlabel('wild-type fold-change:' + r'  $\mathrm{LFC}_{\mathrm{wt}}$')

title = f"n = {sum(data['is_wildtype']==False)} mutants"
ax.set_title(title, loc = 'left')

fig.savefig(FIG_DIR + f"scatterplot_{x_var}_vs_{y_var}_x0={INITIAL_FREQ:.2f}.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)
